In [ ]:
"""
USE AdventureWorksCustomDW

CREATE TABLE FactSales (
	SalesOrderID INT PRIMARY KEY,

	CityKey INT NOT NULL,
	DateKey INT NOT NULL,
	ProductKey INT NOT NULL,
	SpecialOfferKey INT NOT NULL,
	SalesPersonKey INT NOT NULL,

	OrderQty INT NOT NULL,
	SubTotal MONEY NOT NULL
);

"""

'\n\nUSE AdventureWorksCustomDW\n\nCREATE TABLE FactSales (\n\tSalesOrderID INT PRIMARY KEY,\n\n\tCityKey INT NOT NULL,\n\tDateKey INT NOT NULL,\n\tProductKey INT NOT NULL,\n\tSpecialOfferKey INT NOT NULL,\n\tSalesPersonKey INT NOT NULL,\n\n\tOrderQty INT NOT NULL,\n\tSubTotal MONEY NOT NULL\n);\n\nCREATE nonclustered index NI_CityKey ON FactSales (CityKey);\nCREATE nonclustered index NI_DateKey ON FactSales (DateKey);\nCREATE nonclustered index NI_ProductKey ON FactSales (ProductKey);\nCREATE nonclustered index NI_SpecialOfferKey ON FactSales (SpecialOfferKey);\nCREATE nonclustered index NI_SalesPersonKey ON FactSales (SalesPersonKey);\n\n'

In [ ]:
"""
CREATE nonclustered index NI_CityKey ON FactSales (CityKey);
CREATE nonclustered index NI_DateKey ON FactSales (DateKey);
CREATE nonclustered index NI_ProductKey ON FactSales (ProductKey);
CREATE nonclustered index NI_SpecialOfferKey ON FactSales (SpecialOfferKey);
CREATE nonclustered index NI_SalesPersonKey ON FactSales (SalesPersonKey);
"""

In [ ]:
"""
CREATE TABLE DimCity (
    CityKey INT PRIMARY KEY,
    CityName VARCHAR(50)
)

CREATE TABLE DimDate (
    DateKey INT PRIMARY KEY,
    Date DateTime
)

CREATE TABLE DimProduct (
    ProductKey INT PRIMARY KEY,
    ProductName VARCHAR(50),
    ProductColor VARCHAR(20),
    ProductSize VARCHAR(20),
    ProductWeight DECIMAL(8,2),
)

CREATE TABLE DimSpecialOffer (
    SpecialOfferKey INT PRIMARY KEY,
    SpecialOfferType VARCHAR(50),
    SpecialOfferCategory VARCHAR(50)
)

CREATE TABLE DimSalesPerson (
    SalesPersonKey INT PRIMARY KEY,
    SalesPersonName VARCHAR(100),
    SalesPersonGender CHAR(1),
    SalesPersonMaritalStatus CHAR(1),
    SalesPersonHireDate Date
)
"""

In [ ]:
"""
ALTER TABLE FactSales
    ADD CONSTRAINT FK_FactSales_City FOREIGN KEY (CityKey) REFERENCES DimCity(CityKey),
    CONSTRAINT FK_FactSales_Date FOREIGN KEY (DateKey) REFERENCES DimDate(DateKey),
    CONSTRAINT FK_FactSales_Product FOREIGN KEY (ProductKey) REFERENCES DimProduct(ProductKey),
    CONSTRAINT FK_FactSales_SpecialOffer FOREIGN KEY (SpecialOfferKey) REFERENCES DimSpecialOffer(SpecialOfferKey),
    CONSTRAINT FK_FactSales_SalesPerson FOREIGN KEY (SalesPersonKey) REFERENCES DimSalesPerson(SalesPersonKey);
"""

In [ ]:
"""
CREATE TABLE StagingTable (
    SalesOrderID INT UNIQUE NOT NULL, 

    CityID INT NOT NULL,
	CityName NVARCHAR(50),

    DateID INT NOT NULL,
	[Date] DATETIME,

    ProductID INT NOT NULL,
	ProductName NVARCHAR(50),
	ProductColor NVARCHAR(50),
	ProductSize NVARCHAR(50),
	ProductWeight NVARCHAR(50),

	SalesPersonID INT NOT NULL,
	SalesPersonName NVARCHAR(150),
	SalesPersonGender NCHAR(1),
	SalesPersonMaritalStatus NCHAR(1),
	SalesPersonHireDate DATE,

    SpecialOfferID INT NOT NULL,
	SpecialOfferType NVARCHAR(50),
	SpecialOfferCategory NVARCHAR(50),

    OrderQty INT,
    SubTotal MONEY
);
"""

In [ ]:
"""
INSERT INTO AdventureWorksCustomDW.dbo.StagingTable

SELECT 
	ROW_NUMBER() OVER (ORDER BY s.SalesOrderID) AS SalesOrderID,
	ad.City AS CityName,
	s.[OrderDate] AS [Date],

	pp.[Name] AS ProductName,
	pp.Color AS ProductColor,
	pp.Size AS ProductSize,
	pp.[Weight] AS ProductWeight, 

	(ps.FirstName + ' ' + ps.LastName) AS SalesPersonName,
	he.Gender AS SalesPersonGender, 
	he.MaritalStatus AS SalesPersonMaritalStatus,
	he.HireDate AS SalesPersonHireDate, 

	so.[Type] AS SpecialOfferType, 
	so.Category AS SpecialOfferCategory, 

    sd.OrderQty, 
    sd.LineTotal AS SubTotal 

FROM Sales.SalesOrderHeader s
LEFT JOIN Sales.SalesOrderDetail sd ON s.SalesOrderID = sd.SalesOrderID
LEFT JOIN Person.[Address] ad ON s.BillToAddressID = ad.AddressID 
LEFT JOIN Production.[Product] pp ON sd.ProductID = pp.ProductID
LEFT JOIN Sales.SpecialOffer so ON sd.SpecialOfferID = so.SpecialOfferID
LEFT JOIN HumanResources.Employee he ON s.SalesPersonID = he.BusinessEntityID
LEFT JOIN Person.Person ps ON s.SalesPersonID = ps.BusinessEntityID
"""

In [ ]:
"""
TRUNCATE TABLE FactSales

ALTER TABLE FactSales DROP CONSTRAINT FK_FactSales_Date
ALTER TABLE FactSales DROP CONSTRAINT FK_FactSales_City
ALTER TABLE FactSales DROP CONSTRAINT FK_FactSales_SalesPerson
ALTER TABLE FactSales DROP CONSTRAINT FK_FactSales_Product
ALTER TABLE FactSales DROP CONSTRAINT FK_FactSales_SpecialOffer

TRUNCATE TABLE DimDate
TRUNCATE TABLE DimCity
TRUNCATE TABLE DimSalesPerson
TRUNCATE TABLE DimProduct
TRUNCATE TABLE DimSpecialOffer

ALTER TABLE FactSales
ADD CONSTRAINT FK_FactSales_Date FOREIGN KEY (DateKey) REFERENCES DimDate (DateKey),
CONSTRAINT FK_FactSales_City FOREIGN KEY (CityKey) REFERENCES DimCity (CityKey),
CONSTRAINT FK_FactSales_SalesPerson FOREIGN KEY (SalesPersonKey) REFERENCES DimSalesPerson (SalesPersonKey),
CONSTRAINT FK_FactSales_Product FOREIGN KEY (ProductKey) REFERENCES DimProduct (ProductKey),
CONSTRAINT FK_FactSales_SpecialOffer FOREIGN KEY (SpecialOfferKey) REFERENCES DimSpecialOffer (SpecialOfferKey);
"""

In [ ]:
"""
WITH SalesPersonCTE AS (
    SELECT DISTINCT SalesPersonName, SalesPersonGender, SalesPersonMaritalStatus, SalesPersonHireDate
    FROM StagingTable
)

SELECT
    ROW_NUMBER() OVER (ORDER BY SalesPersonName) AS SalesPersonKey,
    SalesPersonName, SalesPersonGender, SalesPersonMaritalStatus, SalesPersonHireDate
FROM SalesPersonCTE;
"""

In [ ]:
"""
SELECT st.SalesOrderID,

	dc.CityKey,
	dt.DateKey,
	dp.ProductKey,
	dso.SpecialOfferKey,
	dsp.SalesPersonKey,

	st.OrderQty,
	st.SubTotal

FROM StagingTable st
LEFT JOIN DimCity dc ON st.CityName = dc.CityName
LEFT JOIN DimDate dt ON st.[Date] = dt.[Date]
LEFT JOIN DimProduct dp ON st.ProductName = dp.ProductName
LEFT JOIN DimSpecialOffer dso ON st.SpecialOfferType = dso.SpecialOfferType
LEFT JOIN DimSalesPerson dsp ON st.SalesPersonName = dsp.SalesPersonName

ORDER BY st.SalesOrderID
"""